In [1]:
import scrapy 
from scrapy.crawler import CrawlerProcess
from scrapy.selector import Selector
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
import re
import requests

In [2]:
class TimviecSpider(scrapy.Spider):
    name = "TimViec"
    start_urls = ["https://www.careerlink.vn/viec-lam/k/php", ]
    custom_settings = {
        "ITEM_PIPELINES": {"__main__.ExtractTimviec": 1},
        "FEED_FORMAT": "csv",
        "FEED_URI": "works.csv"
    }
    
    rules = (Rule(LinkExtractor(allow=(), restrict_xpaths=('//ul[@class="pagination"]/li/a[@class="page-link d-none d-md-block"]',)), callback="parse", follow= True),)
    def parse(self, response):
        container = response.css("ul.list-group.mt-4")
        for res in container.css("li"):
            tieude = res.css("h5.job-name::text")
            link = res.css("a.joblink::attr(href)")
            cty = res.css("a.job-company::text")
            hinh = res.css("div.job-logo > img::attr(src)")
            url = link.get()
            if not "https://" in url:
                url = "https://www.careerlink.vn" + url
            yield {"tieude": tieude, "link": url, "cty": cty.get(), "hinh": hinh.get()}
        
        next_page = response.xpath8('//ul[@class="pagination"]/li/a[@class="page-link d-none d-md-block"]')
        
        if len(next_page) >0:
            next_href =""
            for btn in next_page:
                if btn.xpath("@rel").get() == "next":
                    next_href = btn.xpath("@href").get()
                if next_href != "":
                    if not "https://" in next_href:
                        next_page_url = "https://www.careerlink.vn" + next_href
                    else:
                        next_page_url = next_href
                    requests = scrapy.Requests(url = next_page_url)
                    yield request
        mydata = {}
        mydata["dulieu"] = data
        yield mydata

In [3]:
class ExtractTimviec(object):
    def process_item(self, item, spider):
        tieude = self.remove_html_tags(dict(item)["tieude"])
        link = self.remove_html_tags(dict(item)["link"])
        cty = self.remove_html_tags(dict(item)["cty"])
        hinh = self.remove_html_tags(dict(item)["hinh"])
        self.save_image(hinh)
        return {"hinh": hinh, "tieude": tieude, "link": link, "cty": cty}
    
    def remove_html_tags(self, text):
        html_tags = re.complie("<.*?>")
        content = re.sub(html_tags, "", text)
        content = content.replace("\r", "").replace("\n", "")
        return content.strip()
    
    def save_image(self, url):
        filename = re.search(r"/([\w_-]+[.](jpg|gif|png))$", url)
        if filename:
            with open("image" + filename.group(0), "wb") as f:
                if "http://" in url or "https://" in url:
                    response = requests.get(url)
                    f.write(response.content)
        
        

In [4]:
process = CrawlerProcess()
process.crawl(TimviecSpider)
process.start()

2022-05-11 18:16:43 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-05-11 18:16:43 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)], pyOpenSSL 22.0.0 (OpenSSL 3.0.3 3 May 2022), cryptography 37.0.2, Platform Windows-10-10.0.19044-SP0
2022-05-11 18:16:43 [scrapy.crawler] INFO: Overridden settings:
{}
2022-05-11 18:16:43 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2022-05-11 18:16:43 [scrapy.extensions.telnet] INFO: Telnet Password: bc2521b269e2054c
2022-05-11 18:16:43 [py.warnings] WARNING: C:\Users\tungn\AppData\Local\Programs\Python\Python310\lib\site-packages\scrapy\extensions\feedexport.py:289: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more d